In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Concatenate, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import classification_report
import pandas as pd
from keras.layers import Embedding
from gensim.models import Word2Vec


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
tweet_df = pd.read_csv('/content/drive/MyDrive/tweets.txt', delimiter='\t')
tweet_df

,tweetId,tweetText,userId,imageId(s),username,timestamp,label
0,263046056240115712,¿Se acuerdan de la película: “El día después d...,21226711,sandyA_fake_46,iAnnieM,Mon Oct 29 22:34:01 +0000 2012,fake
1,262995061304852481,@milenagimon: Miren a Sandy en NY! Tremenda i...,192378571,sandyA_fake_09,CarlosVerareal,Mon Oct 29 19:11:23 +0000 2012,fake
2,262979898002534400,"Buena la foto del Huracán Sandy, me recuerda a...",132303095,sandyA_fake_09,LucasPalape,Mon Oct 29 18:11:08 +0000 2012,fake
3,262996108400271360,Scary shit #hurricane #NY http://t.co/e4JLBUfH,241995902,sandyA_fake_29,Haaaaarryyy,Mon Oct 29 19:15:33 +0000 2012,fake
4,263018881839411200,My fave place in the world #nyc #hurricane #sa...,250315890,sandyA_fake_15,princess__natt,Mon Oct 29 20:46:02 +0000 2012,fake
...,...,...,...,...,...,...,...
14272,443231991593304064,@BobombDom *slaps TweetDeck with the PigFish h...,2179310905,pigFish_01,Da_Vault_Hunter,Tue Mar 11 03: 48: 36 +0000 2014,fake
14273,443086239127076865,New Species of Fish found in Brazil or just Re...,254843101,pigFish_01,DjSituation_RC,Mon Mar 10 18: 09: 26 +0000 2014,fake
14274,442978105238753280,What do we call this? #pigFISH http: \/\/t.co\...,2367553228,pigFish_01,Vivo1Vuyo,Mon Mar 10 10: 59: 45 +0000 2014,fake
14275,442753479782989824,Pigfish ? E dopo il pescecane c'è il pesce mai...,603120231,pigFish_01,CosimoTarta,Sun Mar 09 20: 07: 10 +0000 2014,fake


##Filtering the tweets data ,removing labels humor

In [4]:
tweet_df = tweet_df[tweet_df['label'] != 'humor']

# Find rows where 'imgs' contains a comma
mask = tweet_df['imageId(s)'].str.contains(',')

# Drop rows where 'imgs' contains a comma
tweet_df.drop(tweet_df[mask].index, inplace=True)

# Reset the index after processing
tweet_df.reset_index(drop=True, inplace=True)
tweet_real_df = tweet_df[tweet_df['label'] == 'real']
tweet_real_df

tweet_fake_df = tweet_df[tweet_df['label'] == 'fake']
tweet_fake_df

<ipython-input-4-2c2da0f24737>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_df.drop(tweet_df[mask].index, inplace=True)


,tweetId,tweetText,userId,imageId(s),username,timestamp,label
0,263046056240115712,¿Se acuerdan de la película: “El día después d...,21226711,sandyA_fake_46,iAnnieM,Mon Oct 29 22:34:01 +0000 2012,fake
1,262995061304852481,@milenagimon: Miren a Sandy en NY! Tremenda i...,192378571,sandyA_fake_09,CarlosVerareal,Mon Oct 29 19:11:23 +0000 2012,fake
2,262979898002534400,"Buena la foto del Huracán Sandy, me recuerda a...",132303095,sandyA_fake_09,LucasPalape,Mon Oct 29 18:11:08 +0000 2012,fake
3,262996108400271360,Scary shit #hurricane #NY http://t.co/e4JLBUfH,241995902,sandyA_fake_29,Haaaaarryyy,Mon Oct 29 19:15:33 +0000 2012,fake
4,263018881839411200,My fave place in the world #nyc #hurricane #sa...,250315890,sandyA_fake_15,princess__natt,Mon Oct 29 20:46:02 +0000 2012,fake
...,...,...,...,...,...,...,...
11536,443231991593304064,@BobombDom *slaps TweetDeck with the PigFish h...,2179310905,pigFish_01,Da_Vault_Hunter,Tue Mar 11 03: 48: 36 +0000 2014,fake
11537,443086239127076865,New Species of Fish found in Brazil or just Re...,254843101,pigFish_01,DjSituation_RC,Mon Mar 10 18: 09: 26 +0000 2014,fake
11538,442978105238753280,What do we call this? #pigFISH http: \/\/t.co\...,2367553228,pigFish_01,Vivo1Vuyo,Mon Mar 10 10: 59: 45 +0000 2014,fake
11539,442753479782989824,Pigfish ? E dopo il pescecane c'è il pesce mai...,603120231,pigFish_01,CosimoTarta,Sun Mar 09 20: 07: 10 +0000 2014,fake


In [5]:
image_df = pd.read_csv('/content/drive/MyDrive/testimages.txt', delimiter='\t', header=1, names=['image_id', 'image_url', 'annotation', 'event'])

In [6]:
image_df

,image_id,image_url,annotation,event
0,boston_fake_02,http://instagram.com/p/YN7_ThPXrU/,fake,boston
1,boston_fake_03,https://o.twimg.com/2/proxy.jpg?t=HBgeaHR0cDov...,fake,boston
2,boston_fake_04,http://media.tumblr.com/a813460e72a178d8127b50...,fake,boston
3,boston_fake_05,http://i.imgur.com/uxAh4wwh.jpg,fake,boston
4,boston_fake_06,http://www.nowtheendbegins.com/blog/wp-content...,fake,boston
...,...,...,...,...
355,livr_02,https://pbs.twimg.com/media/Bic4fA5CQAAT5KY.jpg,Livr,NaN
356,livr_03,https://pbs.twimg.com/media/BialJuECIAATXE2.jpg,Livr,NaN
357,livr_04,https://pbs.twimg.com/media/BiVisqpCIAAbgju.jpg,Livr,NaN
358,passport_01,https://pbs.twimg.com/media/Bo-CY5ACcAAiRAw.jpg,Passport,NaN


##Filtering the images ,removing labels humor

In [7]:
image_df = image_df[image_df['annotation'] != 'humor']

In [8]:
image_df

,image_id,image_url,annotation,event
0,boston_fake_02,http://instagram.com/p/YN7_ThPXrU/,fake,boston
1,boston_fake_03,https://o.twimg.com/2/proxy.jpg?t=HBgeaHR0cDov...,fake,boston
2,boston_fake_04,http://media.tumblr.com/a813460e72a178d8127b50...,fake,boston
3,boston_fake_05,http://i.imgur.com/uxAh4wwh.jpg,fake,boston
4,boston_fake_06,http://www.nowtheendbegins.com/blog/wp-content...,fake,boston
...,...,...,...,...
355,livr_02,https://pbs.twimg.com/media/Bic4fA5CQAAT5KY.jpg,Livr,NaN
356,livr_03,https://pbs.twimg.com/media/BialJuECIAATXE2.jpg,Livr,NaN
357,livr_04,https://pbs.twimg.com/media/BiVisqpCIAAbgju.jpg,Livr,NaN
358,passport_01,https://pbs.twimg.com/media/Bo-CY5ACcAAiRAw.jpg,Passport,NaN


In [9]:

img_dir = '/content/drive/MyDrive/Medieval2015_DevSet_Images'
folders = os.listdir(img_dir)

folder_tuples = [(folder, folder.lower().replace(' ', '')) for folder in folders]
folder_tuples



[('Hurricane Sandy', 'hurricanesandy'),
 ('BostonMarathon', 'bostonmarathon'),
 ('Sochi Olympics', 'sochiolympics'),
 ('Malaysia Airlines', 'malaysiaairlines'),
 ('Rock Elephant', 'rockelephant'),
 ('Bring Back Our Girls', 'bringbackourgirls'),
 ('Underwater bedroom', 'underwaterbedroom'),
 ('Pig Fish', 'pigfish'),
 ('Livr', 'livr'),
 ('Passport', 'passport'),
 ('Columbian Chemicals', 'columbianchemicals')]

In [10]:
for index, row in image_df.iterrows():
    event = row['event']
    for folder_tuple in folder_tuples:
      image_df.at[index, 'image_dir'] = img_dir+'/'+folder_tuple[0]+'/'+ row['annotation']+'s'
      break
            
image_df

,image_id,image_url,annotation,event,image_dir
0,boston_fake_02,http://instagram.com/p/YN7_ThPXrU/,fake,boston,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
1,boston_fake_03,https://o.twimg.com/2/proxy.jpg?t=HBgeaHR0cDov...,fake,boston,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
2,boston_fake_04,http://media.tumblr.com/a813460e72a178d8127b50...,fake,boston,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
3,boston_fake_05,http://i.imgur.com/uxAh4wwh.jpg,fake,boston,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
4,boston_fake_06,http://www.nowtheendbegins.com/blog/wp-content...,fake,boston,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
...,...,...,...,...,...
355,livr_02,https://pbs.twimg.com/media/Bic4fA5CQAAT5KY.jpg,Livr,NaN,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
356,livr_03,https://pbs.twimg.com/media/BialJuECIAATXE2.jpg,Livr,NaN,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
357,livr_04,https://pbs.twimg.com/media/BiVisqpCIAAbgju.jpg,Livr,NaN,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
358,passport_01,https://pbs.twimg.com/media/Bo-CY5ACcAAiRAw.jpg,Passport,NaN,/content/drive/MyDrive/Medieval2015_DevSet_Ima...


Filtering the tweets that have images

In [11]:
tweet_ids_with_images = set(image_df['image_id'].tolist())


In [12]:
tweet_df = tweet_df.rename({'imageId(s)': 'image_id'}, axis=1)  # new method

In [13]:
tweet_df

,tweetId,tweetText,userId,image_id,username,timestamp,label
0,263046056240115712,¿Se acuerdan de la película: “El día después d...,21226711,sandyA_fake_46,iAnnieM,Mon Oct 29 22:34:01 +0000 2012,fake
1,262995061304852481,@milenagimon: Miren a Sandy en NY! Tremenda i...,192378571,sandyA_fake_09,CarlosVerareal,Mon Oct 29 19:11:23 +0000 2012,fake
2,262979898002534400,"Buena la foto del Huracán Sandy, me recuerda a...",132303095,sandyA_fake_09,LucasPalape,Mon Oct 29 18:11:08 +0000 2012,fake
3,262996108400271360,Scary shit #hurricane #NY http://t.co/e4JLBUfH,241995902,sandyA_fake_29,Haaaaarryyy,Mon Oct 29 19:15:33 +0000 2012,fake
4,263018881839411200,My fave place in the world #nyc #hurricane #sa...,250315890,sandyA_fake_15,princess__natt,Mon Oct 29 20:46:02 +0000 2012,fake
...,...,...,...,...,...,...,...
11536,443231991593304064,@BobombDom *slaps TweetDeck with the PigFish h...,2179310905,pigFish_01,Da_Vault_Hunter,Tue Mar 11 03: 48: 36 +0000 2014,fake
11537,443086239127076865,New Species of Fish found in Brazil or just Re...,254843101,pigFish_01,DjSituation_RC,Mon Mar 10 18: 09: 26 +0000 2014,fake
11538,442978105238753280,What do we call this? #pigFISH http: \/\/t.co\...,2367553228,pigFish_01,Vivo1Vuyo,Mon Mar 10 10: 59: 45 +0000 2014,fake
11539,442753479782989824,Pigfish ? E dopo il pescecane c'è il pesce mai...,603120231,pigFish_01,CosimoTarta,Sun Mar 09 20: 07: 10 +0000 2014,fake


In [14]:
tweet_df = tweet_df[tweet_df['image_id'].isin(tweet_ids_with_images)]

Merging both the daraframes on image dataframe

In [15]:
tweet_df = tweet_df[tweet_df['label'].isin(['fake', 'real'])]
merged_df = tweet_df.merge(image_df, on='image_id')
merged_df=merged_df[['tweetText','image_id','label','annotation','image_dir']]
merged_df.shape

(11509, 5)

In [16]:
# Remove all rows where the label column has the value "humor"
merged_df = merged_df[merged_df['image_id'] != 'boston_fake_10']
# Remove all rows where the label column has the value "humor"
merged_df = merged_df[merged_df['image_id'] != 'sandyB_real_4']
# Remove all rows where the label column has the value "humor"
merged_df = merged_df[merged_df['image_id'] != 'sandyB_real_6']

In [17]:
merged_df.shape

(11500, 5)

In [18]:
merged_df = merged_df.sample(frac=0.35)

In [19]:
tweet_fake_df = merged_df[merged_df['label'] == 'fake']
tweet_real_df = merged_df[merged_df['label'] == 'real']
tweet_fake_df


,tweetText,image_id,label,annotation,image_dir
11330,Courtney Love: I may have found missing Malays...,malaysia_fake_18,fake,fake,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
236,"La foto del día tiene nombre y apellidos , el ...",sandyA_fake_09,fake,fake,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
10088,FBI Hunting for 2 Arab Men seen Holding Bags w...,boston_fake_32,fake,fake,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
413,صورة دخول اعصار ساندي على مدينة نيويورك ..سبحا...,sandyA_fake_09,fake,fake,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
10731,"In Borno/Yobe/Lake Chad?? RT ""@abati1990: Nige...",bringback_fake_02,fake,fake,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
...,...,...,...,...,...
3712,Oh naw sharks in the streets #HurricaneSandy.....,sandyA_fake_05,fake,fake,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
2224,"Shut the fuck up. This was just on the news, s...",sandyA_fake_11,fake,fake,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
4447,demarco53's photo http://t.co/NnRsRm7q #sandy,sandyA_fake_02,fake,fake,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
2558,Shark in the water in Reho. got washed inland ...,sandyA_fake_11,fake,fake,/content/drive/MyDrive/Medieval2015_DevSet_Ima...


In [20]:
tweet_real_df

,tweetText,image_id,label,annotation,image_dir
6984,stay safe east coast friends. #hurricanesandy ...,sandyA_real_33,real,real,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
9384,“@GovChristie: Talking with Pres. Obama about ...,sandyB_real_94,real,real,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
6614,"The #Hudson #River has made #Hoboken, NJ, into...",sandyA_real_18,real,real,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
5943,This shits just fucking sickening... #Crazy #s...,sandyA_real_16,real,real,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
10395,They finally caught his bitch ass!!!! Haha im ...,boston_real_21,real,real,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
...,...,...,...,...,...
9441,This is...unreal. Essentially the only buildin...,sandyB_real_72,real,real,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
7732,Dear mainstream media !!\n#Cuba #USA #Media #S...,sandyB_real_10,real,real,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
5534,RT @raymondjlee: OMG Apparently Ground Zero is...,sandyA_real_04,real,real,/content/drive/MyDrive/Medieval2015_DevSet_Ima...
7179,"Superstorm #Sandy finally# receded, but only a...",sandyA_real_20,real,real,/content/drive/MyDrive/Medieval2015_DevSet_Ima...


In [21]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load(image_path, target_size=(224, 224)):
  # Check if file exists with valid extensions
  valid_extensions = ['.jpg', '.jpeg', '.png','.gif']
  for ext in valid_extensions:
    img_path = os.path.join(image_path + ext)
    if os.path.exists(img_path):
      break
  else:
    return None

  # Load and preprocess image
  img = load_img(img_path, target_size=target_size)
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img /= 255.

  return img

imgNot=[]
image_data = []
 
for index, row in merged_df.iterrows():
  img_path=row['image_dir'] +'/'+row['image_id']
  img = load(img_path, target_size=(224, 224))
  if img is None:
    print(img_path+" is none")
    merged_df = merged_df.drop(merged_df[merged_df['image_id'] == row['image_id']].index)
  else:
    image_data.append(img)

/content/drive/MyDrive/Medieval2015_DevSet_Images/Hurricane Sandy/fakes/malaysia_fake_18 is none
/content/drive/MyDrive/Medieval2015_DevSet_Images/Hurricane Sandy/fakes/boston_fake_32 is none
/content/drive/MyDrive/Medieval2015_DevSet_Images/Hurricane Sandy/reals/boston_real_21 is none
/content/drive/MyDrive/Medieval2015_DevSet_Images/Hurricane Sandy/fakes/bringback_fake_02 is none
/content/drive/MyDrive/Medieval2015_DevSet_Images/Hurricane Sandy/columbianChemicalss/columbianChemicals_fake_01 is none
/content/drive/MyDrive/Medieval2015_DevSet_Images/Hurricane Sandy/fakes/sochi_fake_9 is none
/content/drive/MyDrive/Medieval2015_DevSet_Images/Hurricane Sandy/fakes/bringback_fake_02 is none
/content/drive/MyDrive/Medieval2015_DevSet_Images/Hurricane Sandy/reals/boston_real_24 is none
/content/drive/MyDrive/Medieval2015_DevSet_Images/Hurricane Sandy/fakes/malaysia_fake_18 is none
/content/drive/MyDrive/Medieval2015_DevSet_Images/Hurricane Sandy/fakes/malaysia_fake_15 is none
/content/drive

In [22]:
print(len(image_data))
print(len(merged_df))

3488
3488


In [23]:
image_data = np.vstack(image_data)

In [26]:
word2vec_model = Word2Vec.load('word2vec_model_transfer')

In [33]:
from gensim.models import Word2Vec
import numpy as np

# Tokenize the text data
tokenized_data = [sentence.split() for sentence in merged_df['tweetText']]

# Train a Word2Vec model
model = Word2Vec(tokenized_data, window=5, min_count=1)

# Get the vocabulary
vocabulary = list(model.wv.key_to_index.keys())

# Create the embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((len(vocabulary), embedding_dim))
for i, word in enumerate(vocabulary):
  embedding_vector = model.wv[word]
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [56]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate, Dense

EMBEDDING_DIM = 100 
MAX_SEQUENCE_LENGTH = 100 
MAX_IMAGE_DIM = (224, 224,3) 


# Tokenizing and encoding tweet text
tokenizer = Tokenizer(num_words=MAX_SEQUENCE_LENGTH)
tokenizer.fit_on_texts(merged_df['tweetText']) # df is the dataframe with tweet text
sequences = tokenizer.texts_to_sequences(merged_df['tweetText'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)


# Defining the model
text_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_layer = Embedding(MAX_SEQUENCE_LENGTH,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
embedded_sequences = embedding_layer(text_input)

image_input = Input(shape=(MAX_IMAGE_DIM))
dense = Dense(units=64, activation='relu')(text_input)
conv1 = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(image_input)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flatten = Flatten()(pool2)
concat = Concatenate()([dense, flatten])
output_size = 2
output = Dense(units=output_size, activation='softmax')(concat)
model = Model(inputs=[text_input, image_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert label strings to numeric values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(merged_df['label'])

# Convert numeric labels to binary arrays
labels = to_categorical(labels, num_classes=2)

model.fit([data, image_data], labels, epochs=5, batch_size=32, validation_split=0.2, callbacks=[EarlyStopping(patience=2)])

Found 9197 unique tokens.
Epoch 1/5
88/88 [==============================] - 275s 3s/step - loss: 0.4380 - accuracy: 0.9075 - val_loss: 0.0174 - val_accuracy: 0.9986
Epoch 2/5
88/88 [==============================] - 268s 3s/step - loss: 0.0182 - accuracy: 0.9939 - val_loss: 0.0062 - val_accuracy: 0.9971
Epoch 3/5
88/88 [==============================] - 268s 3s/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 3.4555e-04 - val_accuracy: 1.0000
Epoch 4/5
88/88 [==============================] - 263s 3s/step - loss: 3.4736e-04 - accuracy: 1.0000 - val_loss: 2.6536e-04 - val_accuracy: 1.0000
Epoch 5/5
88/88 [==============================] - 263s 3s/step - loss: 2.9352e-04 - accuracy: 1.0000 - val_loss: 2.0468e-04 - val_accuracy: 1.0000


##VGG16

Building the multimodal model

In [36]:
#loading the test tweet data
test_tweet_df = pd.read_csv('/content/drive/MyDrive/testTweet.txt', delimiter='\t')
test_tweet_df = test_tweet_df[tweet_df['label'] != 'humor']
test_tweet_df


<ipython-input-36-9848f073262a>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_tweet_df = test_tweet_df[tweet_df['label'] != 'humor']


,tweetId,tweetText,userId,imageId(s),username,timestamp,label
0,578854927457349632,kereeen RT @Shyman33: Eclipse from ISS.... htt...,70824972,eclipse_01,peay_s,Fri Mar 20 09:45:43 +0000 2015,fake
1,578874632670953472,Absolutely beautiful! RT @Shyman33: Eclipse fr...,344707006,eclipse_01,JaredUcanChange,Fri Mar 20 11:04:02 +0000 2015,fake
2,578891261353984000,“@Shyman33: Eclipse from ISS.... http://t.co/C...,224839607,eclipse_01,tpjp1231,Fri Mar 20 12:10:06 +0000 2015,fake
3,578846612312748032,Eclipse from ISS.... http://t.co/En87OtvsU6,134543073,eclipse_01,Shyman33,Fri Mar 20 09:12:41 +0000 2015,fake
4,578975333841551360,@ebonfigli: Éclipse vue de l'ISS... Autre chos...,1150728872,eclipse_01,Epimethee_,Fri Mar 20 17:44:11 +0000 2015,fake
...,...,...,...,...,...,...,...
3750,578433150071775232,Un présentateur de la ZDF confesse avoir truqu...,257551211,varoufakis_1,Cdt_Sylvestre,Thu Mar 19 05:49:44 +0000 2015,fake
3751,578433646597656576,"Oh les kleine menteurs ""@CorineBarella: Un pré...",27575883,varoufakis_1,damomarc,Thu Mar 19 05:51:42 +0000 2015,fake
3752,578486910491996160,Este es el programa de ZDF en el que confirman...,2049211,varoufakis_1,javierpascual,Thu Mar 19 09:23:21 +0000 2015,fake
3753,578505023912591360,11.34 - wir haben FAST Mittag ▶ Riesen Verwirr...,262222386,varoufakis_1,aotto1968_2,Thu Mar 19 10:35:20 +0000 2015,fake


In [37]:
# Find rows where 'imgs' contains a comma
mask = test_tweet_df['imageId(s)'].str.contains(',')

# Drop rows where 'imgs' contains a comma
test_tweet_df.drop(test_tweet_df[mask].index, inplace=True)

# Reset the index after processing
test_tweet_df.reset_index(drop=True, inplace=True)

test_tweet_df = test_tweet_df.sample(frac=0.35)

test_tweet_df

,tweetId,tweetText,userId,imageId(s),username,timestamp,label
1397,591995820129517569,#M1Blog Nepal's historic Dharahara Tower colla...,282100036,nepal_25,TMStreetAnalyst,Sat Apr 25 16:02:56 +0000 2015,real
3207,532374834983235584,SYRIA! SYRIAN HERO BOY rescue girl in shootout...,469473252,syrianboy_1,frdi77,Wed Nov 12 03:30:26 +0000 2014,fake
3239,532115700685357056,SYRIA! SYRIAN HERO BOY rescue girl in shootout...,1031001462,syrianboy_1,abedhabib76,Tue Nov 11 10:20:43 +0000 2014,fake
3036,531830780675100672,هرب ولكن هل سينجو اللهم رحمتك وكفالتك لأهل الش...,935861556,syrianboy_1,HarthAl3tibi,Mon Nov 10 15:28:33 +0000 2014,fake
2756,532431408556298240,The Incredible Moment Syrian HERO Boy 'Fakes D...,1860971144,syrianboy_1,JPNoobPlayer,Wed Nov 12 07:15:14 +0000 2014,fake
...,...,...,...,...,...,...,...
2915,532180372767797248,Watch: Syrian 'hero boy' appears to brave snip...,86241025,syrianboy_1,nakhon224,Tue Nov 11 14:37:42 +0000 2014,fake
3254,532648209546354689,Mi è piaciuto un video di @YouTube da @snn_sha...,447065774,syrianboy_1,chaosphantom,Wed Nov 12 21:36:43 +0000 2014,fake
787,591987632260812800,Nepal's historic Dharahara Tower collapses in ...,248774714,nepal_25,GR8socialmedia,Sat Apr 25 15:30:24 +0000 2015,real
3380,532044041852952576,I liked a @YouTube video http://t.co/irGXMlprv...,287214335,syrianboy_1,abozake2030,Tue Nov 11 05:35:59 +0000 2014,fake


In [38]:
#Fetching the path of image directory to a new column image_dir

img_dir = '/content/drive/MyDrive/TestSetImages/TestSetImages'
folders = os.listdir(img_dir)
folders

['Garissa_Attack',
 'Nepal_earthquake',
 'Solar_eclipse',
 'Syrian_boy',
 'Varoufakis_zdf',
 'Samurai_ghost']

In [39]:

for index, row in test_tweet_df.iterrows():
  prefix = row['imageId(s)'].split('_')[0]
  for folder in folders:
    if prefix.lower() in folder.lower():
      test_tweet_df.at[index, 'image_dir'] = img_dir+'/'+folder
      break
            
test_tweet_df
test_tweet_df['imageId(s)'] = test_tweet_df['imageId(s)'].str.rstrip()

test_tweet_df

,tweetId,tweetText,userId,imageId(s),username,timestamp,label,image_dir
1397,591995820129517569,#M1Blog Nepal's historic Dharahara Tower colla...,282100036,nepal_25,TMStreetAnalyst,Sat Apr 25 16:02:56 +0000 2015,real,/content/drive/MyDrive/TestSetImages/TestSetIm...
3207,532374834983235584,SYRIA! SYRIAN HERO BOY rescue girl in shootout...,469473252,syrianboy_1,frdi77,Wed Nov 12 03:30:26 +0000 2014,fake,NaN
3239,532115700685357056,SYRIA! SYRIAN HERO BOY rescue girl in shootout...,1031001462,syrianboy_1,abedhabib76,Tue Nov 11 10:20:43 +0000 2014,fake,NaN
3036,531830780675100672,هرب ولكن هل سينجو اللهم رحمتك وكفالتك لأهل الش...,935861556,syrianboy_1,HarthAl3tibi,Mon Nov 10 15:28:33 +0000 2014,fake,NaN
2756,532431408556298240,The Incredible Moment Syrian HERO Boy 'Fakes D...,1860971144,syrianboy_1,JPNoobPlayer,Wed Nov 12 07:15:14 +0000 2014,fake,NaN
...,...,...,...,...,...,...,...,...
2915,532180372767797248,Watch: Syrian 'hero boy' appears to brave snip...,86241025,syrianboy_1,nakhon224,Tue Nov 11 14:37:42 +0000 2014,fake,NaN
3254,532648209546354689,Mi è piaciuto un video di @YouTube da @snn_sha...,447065774,syrianboy_1,chaosphantom,Wed Nov 12 21:36:43 +0000 2014,fake,NaN
787,591987632260812800,Nepal's historic Dharahara Tower collapses in ...,248774714,nepal_25,GR8socialmedia,Sat Apr 25 15:30:24 +0000 2015,real,/content/drive/MyDrive/TestSetImages/TestSetIm...
3380,532044041852952576,I liked a @YouTube video http://t.co/irGXMlprv...,287214335,syrianboy_1,abozake2030,Tue Nov 11 05:35:59 +0000 2014,fake,NaN


Loading the test images

In [40]:
len(test_tweet_df)
#test_tweet_df.dropna(subset=['image_dir'], inplace=True)
test_tweet_df = test_tweet_df[test_tweet_df['image_dir'].notna()].reset_index(drop=True)

test_tweet_df

,tweetId,tweetText,userId,imageId(s),username,timestamp,label,image_dir
0,591995820129517569,#M1Blog Nepal's historic Dharahara Tower colla...,282100036,nepal_25,TMStreetAnalyst,Sat Apr 25 16:02:56 +0000 2015,real,/content/drive/MyDrive/TestSetImages/TestSetIm...
1,591987563570679808,Nepal's historic Dharahara Tower collapses in ...,622635213,nepal_25,TheSecondMedia,Sat Apr 25 15:30:08 +0000 2015,real,/content/drive/MyDrive/TestSetImages/TestSetIm...
2,592066944678760449,Watch Nepal Earthquake Captured On A Hotel Poo...,2481703728,nepal_17,hhaiya50,Sat Apr 25 20:45:34 +0000 2015,fake,/content/drive/MyDrive/TestSetImages/TestSetIm...
3,592734435503697922,#LetsPrayForNepal #NepalEarthquake http://t.co...,515389435,nepal_02,sendyia,Mon Apr 27 16:57:56 +0000 2015,fake,/content/drive/MyDrive/TestSetImages/TestSetIm...
4,592047099681705984,#NEWS #MASHABLE | Nepal's historic Dharahara t...,402785923,nepal_25,IM_NOT_NAKED,Sat Apr 25 19:26:42 +0000 2015,real,/content/drive/MyDrive/TestSetImages/TestSetIm...
...,...,...,...,...,...,...,...,...
649,591996550538104832,Nepal's historic Dharahara Tower collapses in ...,354807673,nepal_25,MediasFuzz,Sat Apr 25 16:05:51 +0000 2015,real,/content/drive/MyDrive/TestSetImages/TestSetIm...
650,591987904731160576,Nepal's historic Dharahara Tower collapses in ...,2162613295,nepal_25,sosmade,Sat Apr 25 15:31:29 +0000 2015,real,/content/drive/MyDrive/TestSetImages/TestSetIm...
651,591920883918209024,Dharahara Tower Then and Now: A History of Ear...,2958461598,nepal_24,tshering99,Sat Apr 25 11:05:10 +0000 2015,real,/content/drive/MyDrive/TestSetImages/TestSetIm...
652,578895417749950464,Photo: tonyballer: priceofliberty: hugsfromhug...,1468336621,eclipse_01,iwjase,Fri Mar 20 12:26:37 +0000 2015,fake,/content/drive/MyDrive/TestSetImages/TestSetIm...


In [41]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
imgNot=[]
test_image_data = []
for index, row in test_tweet_df.iterrows():
  img_path=row['image_dir'] +'/'+row['imageId(s)']
  img = load(img_path, target_size=(224, 224))
  if img is None:
    print(img_path+" is none")
    test_tweet_df = test_tweet_df.drop(test_tweet_df[test_tweet_df['imageId(s)'] == row['imageId(s)']].index)
  else:
    test_image_data.append(img)

/content/drive/MyDrive/TestSetImages/TestSetImages/Varoufakis_zdf/varoufakis_1 is none
/content/drive/MyDrive/TestSetImages/TestSetImages/Varoufakis_zdf/varoufakis_1 is none
/content/drive/MyDrive/TestSetImages/TestSetImages/Varoufakis_zdf/varoufakis_1 is none
/content/drive/MyDrive/TestSetImages/TestSetImages/Solar_eclipse/eclipse_video_01 is none
/content/drive/MyDrive/TestSetImages/TestSetImages/Varoufakis_zdf/varoufakis_1 is none
/content/drive/MyDrive/TestSetImages/TestSetImages/Varoufakis_zdf/varoufakis_1 is none
/content/drive/MyDrive/TestSetImages/TestSetImages/Varoufakis_zdf/varoufakis_1 is none
/content/drive/MyDrive/TestSetImages/TestSetImages/Varoufakis_zdf/varoufakis_1 is none
/content/drive/MyDrive/TestSetImages/TestSetImages/Varoufakis_zdf/varoufakis_1 is none
/content/drive/MyDrive/TestSetImages/TestSetImages/Varoufakis_zdf/varoufakis_1 is none
/content/drive/MyDrive/TestSetImages/TestSetImages/Varoufakis_zdf/varoufakis_1 is none
/content/drive/MyDrive/TestSetImages/Tes

In [42]:
len(test_image_data)

627

In [43]:
len(test_tweet_df)

627

In [44]:
test_image_data = np.vstack(test_image_data)

In [57]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Convert label strings to numeric values
label_encoder = LabelEncoder()
test_tweet_df['label'] = label_encoder.fit_transform(test_tweet_df['label'])

# Tokenize and encode tweet text
tokenizer = Tokenizer(num_words=MAX_SEQUENCE_LENGTH)
tokenizer.fit_on_texts(test_tweet_df['tweetText'])
sequences = tokenizer.texts_to_sequences(test_tweet_df['tweetText'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
test_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Get the predicted labels for the test data
y_pred = model.predict([test_data, test_image_data])
y_pred = np.argmax(y_pred, axis=1)

# Print the classification report for real tweet
print("Classification Report for Real Data:")
print(classification_report(test_tweet_df['label'], y_pred, labels=[0], target_names=["real"]))

# Print the classification report for fake tweet
print("Classification Report for Fake Data:")
print(classification_report(test_tweet_df['label'], y_pred, labels=[1], target_names=["fake"]))


Found 1962 unique tokens.
20/20 [==============================] - 16s 794ms/step
Classification Report for Real Data:
              precision    recall  f1-score   support

        real       0.68      0.18      0.28       213

   micro avg       0.68      0.18      0.28       213
   macro avg       0.68      0.18      0.28       213
weighted avg       0.68      0.18      0.28       213

Classification Report for Fake Data:
              precision    recall  f1-score   support

        fake       0.69      0.96      0.80       414

   micro avg       0.69      0.96      0.80       414
   macro avg       0.69      0.96      0.80       414
weighted avg       0.69      0.96      0.80       414

